In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
df = pd.read_csv('IMDB Dataset.csv')

df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [6]:
df['sentiment'] = df['sentiment'].apply(lambda x : 1 if x == 'positive' else 0)

df.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
5,"Probably my all-time favorite movie, a story o...",1
6,I sure would like to see a resurrection of a u...,1
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
9,If you like original gut wrenching laughter yo...,1


In [7]:
df['review'][0]
#1 grams with TF-IDF 
#features = bag of 1-grams with sparse matrix of tf-idf value
#will find the weight of each feature, using bag of words representation and
#multiply each one of the tf-idf values by that weight
#sum all that, pass it thru sigmoid = logistic reg model
#linear classification

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
count = CountVectorizer()

docs = np.array(['The sun is shining','The weather is sweet',
        'The sun is shining, the weather is sweet, and one is two'])

In [10]:
bag = count.fit_transform(docs)

In [11]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [12]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [1 3 1 1 1 1 2 1 1]]


In [14]:
from sklearn.feature_extraction.text import TfidfTransformer

In [22]:
tfidf = TfidfTransformer(use_idf=True, norm='l2',smooth_idf=True )

In [24]:
np.set_printoptions(precision = 2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.32 0.56 0.32 0.24 0.24 0.24 0.38 0.32 0.24]]


In [47]:
#Data Preparation
#cleaning tags
df.loc[0, 'review'][150:200]

' me.<br /><br />The first thing that struck me abo'

In [50]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>','', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + \
        ' '.join(emoticons).replace('-', '')
    return text

In [51]:
preprocessor(df.loc[0, 'review'][150:200])

' me the first thing that struck me abo'

In [53]:
df['review']= df['review'].apply(preprocessor)

In [55]:
#Tokenization of Documents
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

In [56]:
def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [60]:
tokenizer_porter('my favorite sport is running')

['my', 'favorit', 'sport', 'is', 'run']

In [61]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Murtaza
[nltk_data]     Kazmi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [63]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a sprinter likes sprinting and sprints every day'
                            ) if w not in stop]

['sprinter', 'like', 'sprint', 'sprint', 'everi', 'day']

In [65]:
#6: Transforming text data into tf-idf vectors
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents = None,lowercase=False,
                       preprocessor = None,
                       tokenizer= tokenizer_porter,
                       use_idf=True,
                       norm ='l2',
                       smooth_idf = True)
y = df.sentiment.values
X = tfidf.fit_transform(df.review)

In [67]:
#Document Classification - Logistic Regression

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.5,
                 shuffle=False)

In [68]:
import pickle
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv = 5,
                          scoring = 'accuracy',
                          random_state = 0,
                          n_jobs = -1,
                          verbose = 3,
                          max_iter=300).fit(X_train, y_train)
saved_model = open('saved_model.sav', 'wb')
pickle.dump(clf, saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  2.3min remaining:  3.5min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.5min finished


In [69]:
#MODEL EVALUATION
filename = 'saved_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))


In [70]:
saved_clf.score(X_test, y_test)

C:\Users\Murtaza Kazmi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:2260: ChangedBehaviorWarning: The long-standing behavior to use the accuracy score has changed. The scoring parameter is now used. This warning will disappear in version 0.22.
  ChangedBehaviorWarning)


0.89472